In [3]:
%matplotlib qt
#to open matplotlib window

import csv
import matplotlib.pyplot as plt
import numpy as np
import datetime
import paramiko
import os
import time
import datetime
from scipy import signal

PORT = 22
USERNAME = "root"
PASSWORD = "root"
REMOTE_PATH = "Pitaya-Tests/" 
REMOTE_FOLDER = "Pitaya-Tests"
SAMPLING_RATE = 125e+6

hostName = "169.254.215.235"
port = 22


def run_acquisition_command(samplesNb, dec,FidNb, FileName, larmorFrequency, excitationDuration, delayRepeat): #voir si file path marche    
    filePath = "mesures/" + FileName
    command = f"cd {REMOTE_FOLDER} && ./Acquisition_axi.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {excitationDuration} {delayRepeat}"
    stdin, stdout, stderr = client.exec_command(command)
    output = stdout.read().decode()
    errors = stderr.read().decode()
    print(f"[CMD-SSH] {command}")
    if output:
        print("[OUTPUT SSH]\n", output)
    if errors:
        print("[ERROR SHH]\n", errors)

def create_file_wdate(nameFile):
    # Créer le dossier local avec timestamp
    now = datetime.datetime.now()
    name_local_file = f"mesures/{nameFile}-{now.strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(name_local_file, exist_ok=True)
    
    return name_local_file 
    
def download_file_sftp(nameRemoteFile,nameRemoteFolder,nameLocalFolder):
    """Télécharge le fichier CSV via SFTP"""
    
    # Téléchargement du fichier
    remote_path = REMOTE_PATH + nameRemoteFolder+'/' + nameRemoteFile
    local_path = os.path.join(nameLocalFolder, nameRemoteFile)
    
    try:
        sftp.get(remote_path, local_path)
        print(f"Téléchargé: {nameRemoteFile}")
    except FileNotFoundError:
        print(f"Fichier non trouvé: {remote_path}")
    except Exception as e:
        print(f"Erreur lors du téléchargement de {nameRemoteFile}: {e}")

def read_file(fileName,folderName):
        
        pathOfFile = os.path.join(folderName,fileName)
        "open the file at the given path and give back the numpy vectors time, voltage"
        voltage_acc = []

        with open(pathOfFile, 'r', encoding='utf-8') as fichier_:
            lecteur = csv.reader(fichier_)

            # Lire et parser l'entête du fichier
            ligne_entete = next(lecteur)
            dsize = int(ligne_entete[0])
            decimation = int(ligne_entete[1])
            nombre_de_FID = int(ligne_entete[2])
            gain = float(ligne_entete[3])
            offset = float(ligne_entete[4])
            nb_bits = int(ligne_entete[5])

            # Initialisation
            voltage = [[] for _ in range(nombre_de_FID)]
            mean = []
            voltage_acc = np.zeros(dsize)  # Initialiser au bon format et taille

            # Lecture et conversion des tensions
            for j in range(nombre_de_FID):
                print(j)
                ligne = next(lecteur)

                signal = []

                # for val_bin in ligne:
                #     #val_bin = int(val_bin)
                #     val = convert_to_volt(val_bin, nb_bits, gain, offset)
                #     signal.append(val)  # pas besoin de np.append ici


                for val in ligne:
                    signal.append(float(val))

                # Convertir la ligne en tableau numpy
                signal = np.array(signal)

                #### Calcul de la moyenne et centrage
                #moyenne = np.mean(signal)
                #signal_centre = signal - moyenne

                # Stocker les données
                voltage[j] = signal
                #mean.append(moyenne)

                # Accumulation du signal total
                voltage_acc += signal

        # Création du tableau temps
        duree_mesure = (dsize * decimation) / SAMPLING_RATE 
        time = np.linspace(0, duree_mesure, dsize, endpoint=False)

        # Calcul de la moyenn et centrage du signal accumulé
        moyenne = np.mean(voltage_acc)
        voltage_acc = voltage_acc - moyenne

        print(f"Fichier {pathOfFile} lu. {len(voltage)} signaux FID chargés.")
        return time, voltage, voltage_acc

def plot_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    
    amountFID = len(voltage_matrix)

    plt.figure(figsize=(12, 7))

    # Affichage des courbes FID
    for w in range(amountFID):
        plt.plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')

    # Affichage du signal accumulé
    plt.plot(time_axis, voltage_accumulated_axis, marker='+', linestyle='-', label='Total', linewidth=2, color='black')

    # Mise en forme du graphique
    plt.title(f'Superposition - {graph_name}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Tension (V)')
    plt.grid(False, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

def subpolts_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    # Supposons que voltage_matrix et time_axis soient déjà définis
    amountFID = len(voltage_matrix)

    # Créer une figure et un ensemble de sous-graphes
    fig, axs = plt.subplots(amountFID, 1, figsize=(12, 7 * amountFID), sharex=True)

    # Affichage des courbes FID sur des sous-graphes séparés
    for w in range(amountFID):
        axs[w].plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')
        axs[w].set_title(f'FID {w+1}')
        axs[w].set_ylabel('Tension (V)')
        axs[w].grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        axs[w].legend(loc='upper right')

    # Mise en forme globale du graphique
    fig.suptitle(f'Superposition - {graph_name}', y=1.02)  # y est légèrement ajusté pour éviter le chevauchement
    plt.xlabel('Temps (s)')
    plt.tight_layout()
    plt.show()

def plot_fourier_transform(time, voltage):
    # Ensure time and voltage are numpy arrays
    time = np.array(time)
    voltage = np.array(voltage)

    # Sampling interval and frequency
    dt = time[1] - time[0]
    fs = 1 / dt

    # Compute FFT
    N = len(voltage)
    fft_values = np.fft.fft(voltage)
    freq = np.fft.fftfreq(N, dt)

    # Keep only the positive frequencies
    #idx = np.where(freq >= 0)
    #freq = freq[idx]
    magnitude = np.abs(fft_values) * 2 / N  # Normalize amplitude

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(freq, magnitude)
    plt.title("Fourier Transform")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_acc_only(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    
    amountFID = len(voltage_matrix)

    plt.figure(figsize=(12, 7))

    # Affichage du signal accumulé
    plt.plot(time_axis, voltage_accumulated_axis, marker='+', linestyle='-', label='Total', linewidth=2, color='black')

    # Mise en forme du graphique
    plt.title(f'Accumulation de {amountFID}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Tension (V)')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    #plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

**Connexion :**

In [4]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostName, username=USERNAME, password=PASSWORD,port=port)
print(f"[INFO] Connexion SSH établie à {hostName}")

transport = paramiko.Transport((hostName, PORT))
transport.connect(username=USERNAME, password=PASSWORD)
sftp = paramiko.SFTPClient.from_transport(transport)
print(f"[INFO] Connexion SFTP établie à {hostName}")

[INFO] Connexion SSH établie à 169.254.215.235
[INFO] Connexion SFTP établie à 169.254.215.235


**Réglages :**

In [35]:
sample_Amount = 424*2               #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1216/2                   #Decimation
aquisiton_Amount = 1              #Nubmer of acquisitons
experiment_name = "test.csv"            #name of the file of the measured tension
larmor_Frequency_Hertz = 24.3417e+6     #Larmor frequency # 24.3417e+6   
excitation_duration_seconds = 25.500e-6     #Excitation time
delay_repeat_seconds = 0.1e+6

N_freq_find = 1
step_freq = 2e+3

total_time = (sample_Amount * decimation)/SAMPLING_RATE
print(f"temps mesuré : {total_time}s")
## print(f"temps total : {total_time*aquisiton_Amount}")

nb_cycles = larmor_Frequency_Hertz*excitation_duration_seconds
print(f"nb cycles burst : {nb_cycles}")

temps_secondes = (total_time+delay_repeat_seconds)*aquisiton_Amount+3
#print(f"temps total secondes: {temps_secondes}")

print(f"temps total :"+str(datetime.timedelta(seconds=temps_secondes*N_freq_find*1e-6)))

temps mesuré : 0.004124672s
nb cycles burst : 620.71335
temps total :0:00:00.100003


**Acquisition simple :**

In [51]:
run_acquisition_command(sample_Amount, 
                        decimation, 
                        aquisiton_Amount, 
                        experiment_name, 
                        larmor_Frequency_Hertz, 
                        excitation_duration_seconds,
                        delay_repeat_seconds)


[CMD-SSH] cd Pitaya-Tests && ./Acquisition_axi.exe 848 608.0 1 mesures/test.csv 24341700.0 2.55e-05 100000.0
[OUTPUT SSH]
 larmor 24341700.000000, duration excitation 0.000025
fichier crée : mesures/test.csv
wait to be filled
Tr pos1: 0x0
ecriture FID 0
Temps d'execution : 0.006534



**Download File unique:**

In [ ]:
nameRemoteFile = experiment_name
nameRemoteFolder = "mesures"
nameLocalFolder = create_file_wdate("test")
download_file_sftp(nameRemoteFile,nameRemoteFolder,nameLocalFolder)

time_array, voltage_array_matrix, voltageAcc_array = read_file(nameRemoteFile,nameLocalFolder)
plot_acc_only(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
#plot_fourier_transform(time_array, voltageAcc_array)

Téléchargé: test.csv
0
Fichier mesures/test-20251106_102500\test.csv lu. 1 signaux FID chargés.


Socket exception: Une connexion existante a dû être fermée par l’hôte distant (10054)
Socket exception: Une connexion existante a dû être fermée par l’hôte distant (10054)


In [ ]:
time_array, voltage_array_matrix, voltageAcc_array = read_file(nameRemoteFile,nameLocalFolder)
plot_acc_only(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)

boucle pour quand ca bug :

In [ ]:
pathOfFile = os.path.join(nameLocalFolder,nameRemoteFile)
"open the file at the given path and give back the numpy vectors time, voltage"
voltage_acc = []

with open(pathOfFile, 'r', encoding='utf-8') as fichier_:
    lecteur = csv.reader(fichier_)

    # Lire et parser l'entête du fichier
    ligne_entete = next(lecteur)
    dsize = int(ligne_entete[0])
    decimation = int(ligne_entete[1])
    nombre_de_FID = 12506
    gain = float(ligne_entete[3])
    offset = float(ligne_entete[4])
    nb_bits = int(ligne_entete[5])

    # Initialisation
    voltage = [[] for _ in range(nombre_de_FID)]
    mean = []
    voltage_acc = np.zeros(dsize)  # Initialiser au bon format et taille

    # Lecture et conversion des tensions
    for j in range(nombre_de_FID):
        ligne = next(lecteur)

        signal = []

        # for val_bin in ligne:
        #     #val_bin = int(val_bin)
        #     val = convert_to_volt(val_bin, nb_bits, gain, offset)
        #     signal.append(val)  # pas besoin de np.append ici


        for val in ligne:
            signal.append(float(val))

        # Convertir la ligne en tableau numpy
        signal = np.array(signal)

        #### Calcul de la moyenne et centrage
        #moyenne = np.mean(signal)
        #signal_centre = signal - moyenne

        # Stocker les données
        voltage[j] = signal
        #mean.append(moyenne)

        # Accumulation du signal total
        voltage_acc += signal

# Création du tableau temps
duree_mesure = (dsize * decimation) / SAMPLING_RATE 
time = np.linspace(0, duree_mesure, dsize, endpoint=False)

# Calcul de la moyenn et centrage du signal accumulé
moyenne = np.mean(voltage_acc)
voltage_acc = voltage_acc - moyenne

print(f"Fichier {pathOfFile} lu. {len(voltage)} signaux FID chargés.")

In [ ]:
plot_acc_only(experiment_name, time, voltage, voltage_acc)
plot_fourier_transform(time, voltage_acc[150:])
time_array = time
voltage_array_matrix = voltage
voltageAcc_array = voltage_acc

**Boucle pour trouver la fréquence :** 

Boucle pour déclencher à la suite des acquisition (aquisiton_axi.c)

In [ ]:
nameLocalFolder = create_file_wdate("FindFreqAuto"+str(N_freq_find)+str(step_freq))
for i in range(N_freq_find):
    experiment_name = "Step_freq"+str(i)
    run_acquisition_command(sample_Amount, 
                        decimation, 
                        aquisiton_Amount, 
                        experiment_name, 
                        larmor_Frequency_Hertz, 
                        excitation_duration_seconds,
                        delay_repeat_seconds)

    nameRemoteFile = experiment_name
    nameRemoteFolder = "mesures"
    download_file_sftp(nameRemoteFile,nameRemoteFolder,nameLocalFolder)
    
    larmor_Frequency_Hertz += step_freq

In [ ]:
nameRemoteFile = experiment_name
nameRemoteFolder = "mesures"
nameLocalFolder = create_file_wdate("FindFreq")
download_file_sftp(nameRemoteFile,nameRemoteFolder,nameLocalFolder)

time_array, voltage_array_matrix, voltageAcc_array = read_file(nameRemoteFile,nameLocalFolder)
plot_acc(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)

In [ ]:
client.close()
# Fermer la connexion
sftp.close()
transport.close()
print("Connexion SFTP fermée")

**Bloc filtrage**

In [ ]:
# Paramètres du signal
fs = SAMPLING_RATE/decimation  # Fréquence d'échantillonnage (Hz)
t = time_array

freq_basse = 500   # Fréquence basse (Hz)
freq_haute = 2500 #Fréquence haute (Hz)
ordre = 3         # Ordre du filtre

voltageAcc_array = voltageAcc_array

butter = signal.butter(ordre,[freq_basse,freq_haute], 
                    btype='bandpass', fs=fs, output='sos')

signal_filtre = signal.sosfilt(butter, voltageAcc_array)


plt.figure(figsize=(10, 4))
plt.plot(t, signal_filtre)
plt.title("Signal filtre")
plt.xlabel("Temps [s]")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

plot_fourier_transform(time_array, signal_filtre)

""" print("Récupération du tableau voltage_acc")
print("Filtrage, Affichage de la FID et de la TF")

b, a = signal.butter(ordre, freq_haute, 
                    btype='lowpass',analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
plt.axvline(freq_haute, color='green') # cutoff frequency
plt.show() """

